# Introducción a las R-CNN (Regions with CNN features)

Las **R-CNN**, o Redes Neuronales Convolucionales basadas en Regiones (del inglés *Regions with Convolutional Neural Network features*), representan un avance fundamental en el campo de la **visión por computadora**, específicamente en la tarea de **detección de objetos**. Antes de su aparición, los métodos para localizar y clasificar objetos en una imagen eran mucho menos precisos.

El enfoque de R-CNN combina de manera ingeniosa dos ideas principales: las **propuestas de región** para localizar objetos y las **redes neuronales convolucionales (CNN)** para identificar qué son esos objetos.

---

## ¿Cómo funciona una R-CNN?

El proceso de una R-CNN se puede dividir en cuatro pasos principales.

![Diagrama del flujo de trabajo de una R-CNN](images/rcnn_logic.png)

#### 1. Generación de Propuestas de Región (Region Proposal)

En lugar de analizar cada posible recuadro en la imagen (lo cual sería computacionalmente muy costoso), el primer paso es generar un número manejable de "propuestas de región" o *bounding boxes* (cajas delimitadoras) que tengan una alta probabilidad de contener un objeto.

- **Algoritmo Clave:** Para esto, R-CNN utiliza un algoritmo llamado **Búsqueda Selectiva (Selective Search)**.
- **Funcionamiento:** La Búsqueda Selectiva agrupa píxeles basándose en color, textura, tamaño y forma para proponer alrededor de 2,000 regiones candidatas por imagen. Estas regiones son de diferentes tamaños y proporciones, intentando cubrir todos los posibles objetos.

#### 2. Extracción de Características con una CNN

Una vez que tenemos las ~2,000 propuestas de región, cada una de ellas se procesa de forma independiente.

- **Deformación (Warping):** Cada región propuesta se deforma (se estira o encoge) para que tenga un tamaño fijo que la red neuronal convolucional (CNN) pueda aceptar como entrada (por ejemplo, 227x227 píxeles).
- **Pase a través de la CNN:** Cada una de estas imágenes deformadas se pasa a través de una CNN pre-entrenada (como AlexNet). El propósito de la CNN no es clasificar la imagen en esta etapa, sino actuar como un **extractor de características**. La salida de una de las últimas capas de la red (generalmente una capa completamente conectada) es un vector de características (por ejemplo, de 4096 dimensiones) que describe el contenido de la región.

#### 3. Clasificación de las Regiones

Con el vector de características para cada una de las ~2,000 regiones, el siguiente paso es decidir qué objeto (si lo hay) contiene cada región.

- **Clasificadores SVM:** R-CNN entrena un clasificador de **Máquinas de Vectores de Soporte (SVM)** para cada clase de objeto (por ejemplo, un SVM para "gato", otro para "perro", etc.).
- **Decisión:** Cada vector de características se pasa a todos los SVMs. El SVM que da una puntuación más alta determina la clase de la región. También se incluye una clase para "fondo" (background) para descartar regiones que no contienen ningún objeto de interés.

#### 4. Refinamiento de la Caja Delimitadora (Bounding Box Regression)

Las propuestas de región de la Búsqueda Selectiva no siempre son perfectas. A menudo, no encajan precisamente alrededor del objeto.

- **Regresión Lineal:** Para solucionar esto, R-CNN entrena un modelo de **regresión lineal** específico para cada clase de objeto.
- **Ajuste Fino:** Este modelo aprende a ajustar las coordenadas (x, y, ancho, alto) de la caja propuesta para que se alinee mejor con los límites reales del objeto detectado.

---

## Limitaciones de la R-CNN Original

Aunque fue revolucionaria, la arquitectura R-CNN original tenía desventajas significativas que motivaron el desarrollo de versiones mejoradas:

1.  **Extremadamente Lenta:** El mayor problema era su velocidad. Pasar ~2,000 regiones por la CNN, una por una, es un proceso muy lento y redundante. El tiempo de inferencia para una sola imagen podía ser de hasta 40-50 segundos.
2.  **Proceso de Entrenamiento Complejo:** El entrenamiento era un proceso de múltiples etapas y muy costoso en tiempo y recursos. Había que entrenar la CNN, luego los clasificadores SVM y finalmente los regresores de las cajas delimitadoras por separado.
3.  **Almacenamiento Masivo:** Se requería almacenar las características extraídas de cada región para cada imagen del conjunto de datos, lo que ocupaba una gran cantidad de espacio en disco.

## El Legado: Fast R-CNN y Faster R-CNN

Estas limitaciones llevaron al desarrollo de sucesores directos que optimizaron drásticamente el proceso:

- **Fast R-CNN:** Realiza la extracción de características de la CNN **una sola vez sobre la imagen completa**. Luego, proyecta las propuestas de región sobre el mapa de características resultante. Esto evita los miles de pases redundantes por la CNN, acelerando el proceso enormemente.
- **Faster R-CNN:** Va un paso más allá e integra la generación de propuestas de región **dentro de la propia red neuronal** (con un componente llamado Region Proposal Network o RPN), eliminando la dependencia del algoritmo lento de Búsqueda Selectiva. Esto creó un sistema de detección de objetos unificado y mucho más rápido.